In [3]:
from pyspark.sql.functions import col

# ============================================================
# 0. Configurações (Vigorosas e Simples)
# ============================================================
# Desligamos a "mágica" de escrita do Fabric.
# Para volumes pequenos de saída ("milhares de linhas"),
# queremos controle total para gerar apenas 1 arquivo.
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "false")

# Mantemos o AQE ligado para otimizações gerais de leitura
spark.conf.set("spark.sql.adaptive.enabled", "true")

# Caminho físico
path_destino = (
    "abfss://ws_departamento_pessoal@onelake.dfs.fabric.microsoft.com/"
    "lk_departamento_pessoal.Lakehouse/Tables/tab_gold_dim_funcionario_ultima_posicao"
)

# ============================================================
# 1. Ler e Filtrar (O Funil)
# ============================================================
# Leitura de MILHÕES de linhas
df_historico = spark.table("tab_gold_dim_funcionario_historico")

# O Filtro reduz para MILHARES de linhas
df_filtrado = df_historico.filter(col("RK") == 1)

# ============================================================
# 2. ESTRATÉGIA FUNIL (Coalesce)
# ============================================================
# Como saímos de Milhões -> Milhares, não usamos 'repartition' (shuffle).
# Usamos 'coalesce(1)' para fundir as partições vazias e escrever
# o resultado pequeno em UM único arquivo limpo.
# Isso é mais rápido e remove o "falso skew" de escrita.
df_otimizado = df_filtrado.coalesce(1)

# ============================================================
# 3. Escrever
# ============================================================
(
    df_otimizado.write
        .format("delta")
        .mode("overwrite")
        .option("overwriteSchema", "true")
        .save(path_destino)
)

print("Gravação otimizada (Funil) concluída com sucesso.")

StatementMeta(, 26326a2d-332f-4207-b21d-216eedb7e0e0, 5, Finished, Available, Finished)

Gravação otimizada (Funil) concluída com sucesso.
